In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#mlat = np.arange(50, 89, 1) #30 intervals between 53.1° to 89.7°
#mlt= np.arange(1, 360, 1) #80 intervals between 1.6° to 357.6°

#AMPERE
#mlat = np.linspace(40,90,50) #50 intervals between 40° to 90°
# mlt = np.linspace(0,360, 24) #24 intervals between 0° to 360°

mlat = np.linspace(0,90,90) #50 intervals between 40° to 90°
mlt = np.linspace(0,360,360) #24 intervals between 0° to 360°

In [83]:
1 * (np.pi * 6371.008 / 180)

111.19506627089888

In [80]:
def calculate_delta(lat1, lon1, lat2, lon2):
    radius = 6371.008
    
    #solution 1 in co-latitude
    #dlat = np.radians((90-lat1) - (90-lat2))
    #dlon = np.radians(lon2 - lon1)
    #delta_x = dlat * radius
    #delta_y = dlon * radius * np.sin(np.radians(90-lat1))
    
    #solution 2 in latitude
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    delta_x = dlat * radius
    delta_y = dlon * radius * np.cos(np.radians(lat1))
    
    return delta_x, delta_y

In [81]:
np.sin(np.deg2rad(1)), np.sin(np.deg2rad(89))

(0.01745240643728351, 0.9998476951563913)

In [82]:
areas = np.zeros((89, 359))
#reas = np.zeros((49, 23))

for i in range(len(mlat) - 1):
    for j in range(len(mlt) - 1):
        lat1, lat2 = mlat[i], mlat[i + 1]
        lon1, lon2 = mlt[j], mlt[j + 1]
        
        delta_x, delta_y = calculate_delta(lat1, lon1, lat2, lon2)
        
        area = delta_x * delta_y
        areas[i, j] = area

areas

array([[12538.09600704, 12538.09600704, 12538.09600704, ...,
        12538.09600704, 12538.09600704, 12538.09600704],
       [12536.14324168, 12536.14324168, 12536.14324168, ...,
        12536.14324168, 12536.14324168, 12536.14324168],
       [12530.28555387, 12530.28555387, 12530.28555387, ...,
        12530.28555387, 12530.28555387, 12530.28555387],
       ...,
       [  663.55934878,   663.55934878,   663.55934878, ...,
          663.55934878,   663.55934878,   663.55934878],
       [  442.48776268,   442.48776268,   442.48776268, ...,
          442.48776268,   442.48776268,   442.48776268],
       [  221.27834468,   221.27834468,   221.27834468, ...,
          221.27834468,   221.27834468,   221.27834468]])

In [ ]:
def weighted_loss(prediction, target, weights, dampening=1):

    weights = weights**dampening #dampen weighted err
    weights = weights / np.min(weights)  #normalize weights
    
    loss = prediction - target #MAE
    weighted_loss = loss * weights 

    return weighted_loss

target = np.ones((49, 23))
prediction = target + 0.1
weights = np.flipud(areas)

weight_loss = weighted_loss(prediction, target, weights)
weight_loss.min(), weight_loss.max()

In [ ]:
# Define common parameters
theta = np.linspace(0, 360,23) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(50, 90, 49)
shrink = .75

# Plot true values
#abs_error = np.flipud(abs_error) # flip the array upside down

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6,5))

c_true = ax.contourf(theta_rad, r, np.flipud(weight_loss),  cmap='viridis_r', extend='both', levels=15, zorder=0)
ax.set_yticks([0, 10, 20, 30, 40])
ax.set_yticklabels(["90°", "80°", "70°", "60°", "50° MLAT"], color='red')
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 \nMidday", "15", "18 \nDusk"])
ax.set_title(f'Weighting Error Function (Dummy Data)')
plt.colorbar(c_true, ax=ax, label=' MAE (weighted) [-]', shrink=shrink, pad=0.15, orientation='horizontal')
ax.grid(True, linestyle='-', alpha=0.8)
#plt.tight_layout()
plt.savefig('/Users/sr2/My Drive/Career/Employment/Current/NIPR/Research/Assimilation/plots/weighted_MAE_AMP.png', dpi=400, bbox_inches='tight')

In [ ]:
# Define common parameters
#r = 90 - np.linspace(53.1, 89.7, 29)
#shrink = .7

theta = np.linspace(0, 360, 23) - 90
theta_rad = theta / 360 * 2 * np.pi
r = 90 - np.linspace(40, 90, 49)
shrink = .6

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6, 5))

c_true = ax.contourf(theta_rad, r, areas, cmap='cividis_r', extend='both', levels=8, zorder=0)
ax.set_ylim([0, 39])
ax.set_yticks([0, 10, 20, 30, 40])
ax.set_yticklabels(["90°", "80°", "70°", "60°", "50° MLAT"], color='red')
ax.set_xlim([-np.pi, np.pi])
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 \nMidday", "15", "18 \nDusk"])
a_min = int(areas.min())
a_max = int(areas.max())
ax.set_title(f'Area (AMPERE) \n Min: {a_min} km, Max: {a_max} km', linespacing=1.3)
km2 = r'$\mathrm{km^2}$'
plt.colorbar(c_true, ax=ax, label=f'Area [{km2}]', shrink=shrink, pad=0.15, orientation='vertical')
ax.grid(True, linestyle='-', alpha=0.3)
#plt.tight_layout()
plt.savefig('/Users/sr2/My Drive/Career/Employment/Current/NIPR/Research/Assimilation/plots/Area_AMP.png',
            dpi=400, bbox_inches='tight')